# ```sampley``` exemplar: the grid approach
Before going through this exemplar, please consult the _Introduction to sampley exemplars_ (```intro.ipynb```).
<br>This exemplar illustrates an application of the grid approach to data contained within two files: one containing survey tracks (```sections.gpkg```) and one containing sightings data (```sightings.gpkg```).

## Setup

### Import the package

In [1]:
from sampley import *

### Set the input folder
To run this exemplar, download the mock data files, put them in a folder, and set the path to the folder below.

In [2]:
input_folder = './input/'

### Set the output folder
To run this exemplar, make a folder to save the outputs in and set the path to the folder below.

In [3]:
output_folder = './output/'

## Stage 1
In Stage 1, we import two files (```sightings.gpkg``` and ```sections.gpkg```) and from them make a ```DataPoints``` and a ```Sections``` object, respectively.

In [4]:
u_sightings = DataPoints.from_file(
    filepath=input_folder+'sightings.gpkg',
    crs_working='EPSG:32619',
    datetime_col='datetime',
    tz_input='UTC-05:00'
)

Success: file successfully input.
Success: reprojected to CRS 'EPSG:32619'
Success: the column 'datetime' successfully reformatted to datetimes.
Success: the timezone of column 'datetime' successfully set to 'UTC-05:00'.
Success: datapoint IDs successfully generated.


In [5]:
u_sections = Sections.from_file(
    filepath=input_folder+'sections.gpkg',
    crs_working='EPSG:32619',
    datetime_col='datetime_beg',
    tz_input='UTC-05:00'
)

Success: file successfully input.
Success: reprojected to CRS 'EPSG:32619'
Success: the column 'datetime_beg' successfully reformatted to datetimes.
Success: the timezone of column 'datetime_beg' successfully set to 'UTC-05:00'.
Note: column 'datetime_beg' renamed to 'datetime'.
Success: section IDs successfully generated.


## Stage 2
In Stage 2, we use the ```Sections``` object to make a ```Periods``` object containing periods of 8 days and a ```Cells``` object containing hexagonal cells with 5000 m sides.

In [6]:
u_periods = Periods.delimit(
    extent=u_sections,
    unit='day',
    num=8)

In [7]:
u_cells = Cells.delimit(
    extent=u_sections,
    var='hexagonal',
    side=5000)

## Stage 3
In Stage 3, we, firstly, make two ```Samples``` objects:
- one from the ```DataPoints``` object containing sightings data, the ```Periods``` objects, and the ```Cells``` object
- one from the ```Sections``` object, the ```Periods``` objects, and the ```Cells``` object

Then, we merge these two ```Samples``` objects to make our final ```Samples``` object.

In [8]:
u_samples_sightings = Samples.grid(
    datapoints=u_sightings,
    cells=u_cells,
    periods=u_periods,
    cols={'individuals': 'sum'})

In [9]:
u_samples_effort = Samples.grid_se(
    sections=u_sections,
    cells=u_cells,
    periods=u_periods)

In [10]:
u_samples = Samples.merge(
    sightings=u_samples_sightings,
    effort=u_samples_effort)


Note: samples generated with the grid approach


## Output
Finally, we save the final ```Samples``` object to the output folder.

In [11]:
u_samples.save(
    folder=output_folder,
    filetype='csv'
)

It should be saved as a CSV with the same name as the ```Samples``` object:

In [12]:
u_samples.name

'samples-sightings+effort-x-cells-h5000m-x-periods-8d'

And it should contain the samples:

In [13]:
u_samples.samples

,cell_id,polygon,centroid,period_id,datetime_beg,datetime_mid,datetime_end,individuals,se_length
0,c013-h5000m,"POLYGON ((604018.668 4701798.087, 608348.795 4...",POINT (604018.668 4696798.087),p2019-02-02-8d,2019-02-02 00:00:00-05:00,2019-02-05 23:59:59-05:00,2019-02-09 23:59:59-05:00,NaN,6000.522001
1,c014-h5000m,"POLYGON ((612678.922 4701798.087, 617009.049 4...",POINT (612678.922 4696798.087),p2019-02-02-8d,2019-02-02 00:00:00-05:00,2019-02-05 23:59:59-05:00,2019-02-09 23:59:59-05:00,4.0,8662.115497
2,c015-h5000m,"POLYGON ((621339.176 4701798.087, 625669.303 4...",POINT (621339.176 4696798.087),p2019-02-02-8d,2019-02-02 00:00:00-05:00,2019-02-05 23:59:59-05:00,2019-02-09 23:59:59-05:00,2.0,8661.560380
3,c016-h5000m,"POLYGON ((629999.43 4701798.087, 634329.557 46...",POINT (629999.43 4696798.087),p2019-02-02-8d,2019-02-02 00:00:00-05:00,2019-02-05 23:59:59-05:00,2019-02-09 23:59:59-05:00,NaN,8662.595905
4,c017-h5000m,"POLYGON ((638659.684 4701798.087, 642989.811 4...",POINT (638659.684 4696798.087),p2019-02-02-8d,2019-02-02 00:00:00-05:00,2019-02-05 23:59:59-05:00,2019-02-09 23:59:59-05:00,NaN,8662.002385
...,...,...,...,...,...,...,...,...,...
96,c128-h5000m,"POLYGON ((560717.398 4746798.087, 565047.525 4...",POINT (560717.398 4741798.087),p2019-01-25-8d,2019-01-25 00:00:00-05:00,2019-01-28 23:59:59-05:00,2019-02-01 23:59:59-05:00,NaN,8661.000942
97,c129-h5000m,"POLYGON ((569377.652 4746798.087, 573707.779 4...",POINT (569377.652 4741798.087),p2019-01-25-8d,2019-01-25 00:00:00-05:00,2019-01-28 23:59:59-05:00,2019-02-01 23:59:59-05:00,NaN,8661.175721
98,c130-h5000m,"POLYGON ((578037.906 4746798.087, 582368.033 4...",POINT (578037.906 4741798.087),p2019-01-25-8d,2019-01-25 00:00:00-05:00,2019-01-28 23:59:59-05:00,2019-02-01 23:59:59-05:00,1.0,6386.031468
99,c139-h5000m,"POLYGON ((655980.192 4746798.087, 660310.319 4...",POINT (655980.192 4741798.087),p2019-02-02-8d,2019-02-02 00:00:00-05:00,2019-02-05 23:59:59-05:00,2019-02-09 23:59:59-05:00,NaN,9157.070699
